In [ ]:
# 📦 Imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout


In [ ]:
# 📊 Load Data
data = pd.read_csv("Google_Stock_Price_Train.csv")
data.head()
# 🧾 Data Info
data.info()


In [ ]:
# 📌 Prepare Dataset
data_set = data.loc[:, ["Open"]].values
train = data_set[:len(data_set) - 50]
test = data_set[len(train):]
train = train.reshape(train.shape[0],1)
train.shape


In [ ]:
# 🔄 Normalize
scaler = MinMaxScaler(feature_range = (0,1))
train_scaler = scaler.fit_transform(train)

plt.plot(train_scaler)
plt.title("Scaled Training Data")
plt.show()


In [ ]:
# ⏳ Prepare Sequences
X_train = []
Y_train = []
timesteps = 50

for i in range(timesteps, len(train_scaler)):
    X_train.append(train_scaler[i - timesteps:i, 0])
    Y_train.append(train_scaler[i,0])

X_train, Y_train = np.array(X_train), np.array(Y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [ ]:
# 🧠 Build RNN
regressor = Sequential()
regressor.add(SimpleRNN(units = 50, activation = "tanh", return_sequences = True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(SimpleRNN(units = 50, activation = "tanh", return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(SimpleRNN(units = 50, activation = "tanh", return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(SimpleRNN(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))

regressor.compile(optimizer = "adam", loss = "mean_squared_error")
regressor.fit(X_train, Y_train, epochs = 250, batch_size = 16)


In [ ]:
# 🧪 Test Predictions
inputs = data_set[len(data_set) - len(test) - timesteps:]
inputs = scaler.transform(inputs)

X_test = []
for i in range(timesteps, inputs.shape[0]):
    X_test.append(inputs[i - timesteps:i, 0])

X_test_rnn = np.array(X_test)
X_test_rnn = np.reshape(X_test_rnn, (X_test_rnn.shape[0], X_test_rnn.shape[1], 1))
predicted_stock_price = regressor.predict(X_test_rnn)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)


In [ ]:
# 📈 Plot Results
plt.plot(test, color = "green", label = "Real Google Stock Price")
plt.plot(predicted_stock_price, color = "blue", label = "Predicted Google Stock Price")
plt.title("Google Stock Price Prediction")
plt.xlabel("Time")
plt.ylabel("Google Stock Price")
plt.legend()
plt.show()
